In [1]:
%AddDeps org.apache.spark spark-sql-kafka-0-10_2.11 2.4.5 --transitive

Marking org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 for download
Obtained 12 files


In [3]:
import org.apache.spark._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._

In [4]:
val spark = SparkSession.builder().appName("StreamingModel")
            .master("spark://spark:7077").getOrCreate()

spark = org.apache.spark.sql.SparkSession@651641ff


org.apache.spark.sql.SparkSession@651641ff

In [5]:
val dataframe = spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka1:29092")
    .option("subscribe", "Spectra")
    .option("startingOffsets", "earliest")
    .load()

dataframe = [key: binary, value: binary ... 5 more fields]


[key: binary, value: binary ... 5 more fields]

In [10]:
val output = dataframe
            .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
            .as[(String, String)]
            .writeStream
            .outputMode("append")
            .format("memory")
            .queryName("spectra")
            .start()

output = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5bf1623


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5bf1623

In [12]:
def explode_JSON(table: DataFrame, column_name: String): DataFrame = {
    val JSONschema: StructType = spark.read.json(table.select(column_name).as[String]).schema
    table.withColumn("JSON", from_json(col(column_name), JSONschema)).select("JSON.*")
}

val stringToArray = udf((b: String) => b.substring(1, b.length()-1).split(","))

val trimString = udf((b: String) => b.trim())

def process_dataframe(df: DataFrame) = {
    val exploded = explode_JSON(df, "value").select("dataRow.*").drop("spectraSetOID")
    exploded.na.drop("all")
                .withColumn("continuum", stringToArray(col("continuum")).cast(ArrayType(DoubleType, true)))
                .withColumn("dec", col("dec").cast(DoubleType))
                .withColumn("fiber", col("fiber").cast(LongType))
                .withColumn("mjd", col("mjd").cast(LongType))
                .withColumn("plate", col("plate").cast(LongType))
                .withColumn("ra", col("ra").cast(DoubleType))
                .withColumn("size", col("size").cast(LongType))
                .withColumn("spectrum", stringToArray(col("spectrum")).cast(ArrayType(DoubleType, true)))
                .withColumn("subtype", trimString(col("subtype")))
                .withColumn("type", trimString(col("type")))
                .withColumn("z", col("z").cast(DoubleType))
                .withColumn("zerr", col("zerr").cast(DoubleType))
}
    

stringToArray = UserDefinedFunction(<function1>,ArrayType(StringType,true),Some(List(StringType)))
trimString = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


explode_JSON: (table: org.apache.spark.sql.DataFrame, column_name: String)org.apache.spark.sql.DataFrame
process_dataframe: (df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [16]:
val spectra = process_dataframe(spark.sql("select * from spectra"))

spectra = [continuum: array<double>, dec: double ... 11 more fields]


[continuum: array<double>, dec: double ... 11 more fields]

## Pipeline

Trzeba rozdzielić tablice spectrum i continuum na osobne kolumny.

In [45]:
val continuum_size = spectra.select(size($"continuum")).first()

continuum_size = [4096]


[4096]

In [48]:
val spectrum_size = spectra.select(size($"spectrum")).first()

spectrum_size = [4096]


lastException: Throwable = null


[4096]

In [82]:
val columns = (0 until 4096).map(r => spectra.col("spectrum").getItem(r)) ++
              (0 until 4096).map(r => spectra.col("continuum").getItem(r)) ++
              spectra.drop("spectrum", "continuum").columns.map(col)

columns = Vector(spectrum[0], spectrum[1], spectrum[2], spectrum[3], spectrum[4], spectrum[5], spectrum[6], spectrum[7], spectrum[8], spectrum[9], spectrum[10], spectrum[11], spectrum[12], spectrum[13], spectrum[14], spectrum[15], spectrum[16], spectrum[17], spectrum[18], spectrum[19], spectrum[20], spectrum[21], spectrum[22], spectrum[23], spectrum[24], spectrum[25], spectrum[26], spectrum[27], spectrum[28], spectrum[29], spectrum[30], spectrum[31], spectrum[32], spectrum[33], spectrum[34], spectrum[35], spectrum[36], spectrum[37], spectrum[38], spectrum[39], spectrum[40], spectrum[41], spectrum[42], spectrum[43], spectrum[44], spectrum[45], spectrum[46], spectrum[47], spectrum[48], spectrum[49], spectrum[50], spectrum...


Vector(spectrum[0], spectrum[1], spectrum[2], spectrum[3], spectrum[4], spectrum[5], spectrum[6], spectrum[7], spectrum[8], spectrum[9], spectrum[10], spectrum[11], spectrum[12], spectrum[13], spectrum[14], spectrum[15], spectrum[16], spectrum[17], spectrum[18], spectrum[19], spectrum[20], spectrum[21], spectrum[22], spectrum[23], spectrum[24], spectrum[25], spectrum[26], spectrum[27], spectrum[28], spectrum[29], spectrum[30], spectrum[31], spectrum[32], spectrum[33], spectrum[34], spectrum[35], spectrum[36], spectrum[37], spectrum[38], spectrum[39], spectrum[40], spectrum[41], spectrum[42], spectrum[43], spectrum[44], spectrum[45], spectrum[46], spectrum[47], spectrum[48], spectrum[49], spectrum[50], spectrum...

In [84]:
val exploded_spectra = spectra.select(columns: _*)

exploded_spectra = [spectrum[0]: double, spectrum[1]: double ... 8201 more fields]


[spectrum[0]: double, spectrum[1]: double ... 8201 more fields]

In [9]:
output.stop()

lastException: Throwable = null
